In [1]:
import pandas as pd
import os
from pathlib import Path
import scipy
import pickle
import scipy.stats
from numpy import *
import numpy as np
from numpy import linalg as la
from sklearn.metrics import mean_squared_error
root = Path(".")

In [2]:
rating_data = pd.read_csv('ml-1m/ratings.dat', header=None, sep='::')
rating_data.columns = ['UserID', 'ItemID', 'Rating', 'Timestamp']
rating_data.drop(columns=['Timestamp'], axis=1, inplace=True)

D:\Anaconda\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [3]:
rating_data

,UserID,ItemID,Rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
...,...,...,...
1000204,6040,1091,1
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4


In [4]:
item_cnt = max(rating_data['ItemID']) + 1
users_cnt = max(rating_data['UserID']) + 1
item_cnt, users_cnt

(3953, 6041)

In [5]:
rating_train = rating_data.sample(frac=0.8, random_state=200)
rating_test = rating_data.drop(rating_train.index)

In [6]:
rating_test

,UserID,ItemID,Rating
0,1,1193,5
2,1,914,3
8,1,594,4
9,1,919,4
13,1,2918,4
...,...,...,...
1000201,6040,1080,4
1000202,6040,1089,4
1000204,6040,1091,1
1000207,6040,1096,4


In [7]:
matrix = np.zeros(shape=(users_cnt, item_cnt))
matrix.shape

(6041, 3953)

In [8]:
for row in rating_data.itertuples():
    matrix[row.UserID][row.ItemID] = row.Rating
actual_matrix = matrix
actual_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 3., 0., ..., 0., 0., 0.]])

In [9]:
def topKPrecision(actual_mat,pred_mat,test_data,K):
    pred_data = []
    for row in test_data.itertuples():
        pred_data.append((row.UserID,row.ItemID,pred_mat[row.UserID,row.ItemID]))
    sorted(pred_data,key=lambda x: x[-1])
    pred_data.reverse()
    tp = 0
    for uid,iid,pr in pred_data[:K]:
        if actual_mat[uid,iid] >= 3.5:
            tp += 1
    return tp/K

In [10]:
my_path = root / "Pickled_files" / "pred_matrix_collab"
dbfile = open(my_path, 'rb')     
pred_matrix = pickle.load(dbfile)
dbfile.close()

In [11]:
pred_matrix

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 4.8205123 , 4.39766519, ..., 4.70378116, 4.73979053,
        4.68459535],
       [0.        , 4.13735754, 3.        , ..., 3.27905294, 3.47972922,
        3.34977903],
       ...,
       [0.        , 4.60885315, 4.16276878, ..., 4.31833435, 4.38061175,
        4.28871652],
       [0.        , 3.51136871, 3.41617474, ..., 3.61797934, 3.6076365 ,
        3.56541669],
       [0.        , 3.        , 3.56061577, ..., 3.52780734, 3.44930387,
        3.49752001]])

In [18]:
topKPrecision(actual_matrix,pred_matrix,rating_test,10)

0.8

In [19]:
my_path = root / "Pickled_files" / "SVD"
dbfile = open(my_path, 'rb')     
pred_matrix = pickle.load(dbfile)
dbfile.close()

In [20]:
topKPrecision(actual_matrix,pred_matrix,rating_test,10)

0.8

In [14]:
my_path = root / "Pickled_files" / "SVD90"
dbfile = open(my_path, 'rb')     
pred_matrix = pickle.load(dbfile)
dbfile.close()

In [17]:
topKPrecision(actual_matrix,pred_matrix,rating_test,10)

0.8